# Установка библиотек

In [50]:
import pandas as pd #для работы с таблицами

In [51]:
%pip install scipy

Note: you may need to restart the kernel to use updated packages.


In [52]:
%pip install statsmodels

Note: you may need to restart the kernel to use updated packages.


-----

_____

# Домашнее задание

Используя информацию по мидиям Петербурга и Магадана, которые представлены в виде двух массивов petersburg и magadan в разделе ANOVA тест, проверьте данные на нормальность и на наличие корреляции. Для этого объедините 2 массива в DataFrame. Обоснуйте выбор теста на корреляцию. Сделайте вывод по гипотезе.
 
 
Также ответьте на вопрос:
Какова разница в среднем размере мидии в зависимости от города-производителя. Обоснуйте выбор теста. Сделайте вывод по гипотезе.


# ОБЪЕДИНЯЕМ ДВА МАССИВА В DataFrame

Массивы с информацией о мидиях, произведенных в Санкт-Петерурге и Магадане имеют разную длинну. Для проведения сравнительного анализа приведем массивы к одинаковой длинне путем добавления в конце более короткого массива медианного значения.

In [53]:
def my_median(sample): #функция для вычисления медианы
    n = len(sample)
    index = n // 2
     # если нечетное количество элементов
    if n % 2:
        return sorted(sample)[index]
    # если четное количество элементов
    return sum(sorted(sample)[index - 1:index + 1]) / 2

In [54]:
spb_ls = [0.0974, 0.1352, 0.0817, 0.1016, 0.0968, 0.1064, 0.105] #массив с информацией по Санкт-Петербургу
magadan_ls = [0.1033, 0.0915, 0.0781, 0.0685, 0.0677, 0.0697, 0.0764, 0.0689] #массив с информацией по Магадану
spb_ls.append(round(my_median(spb_ls),4))#добавляем медианное значение в массив с информацией по Санкт-Петербургу

In [55]:
#создаем DataFrame для сравнительного анализа
midia_df = pd.DataFrame({'petersburg': spb_ls, 'magadan': magadan_ls})
midia_df

,petersburg,magadan
0,0.0974,0.1033
1,0.1352,0.0915
2,0.0817,0.0781
3,0.1016,0.0685
4,0.0968,0.0677
5,0.1064,0.0697
6,0.1050,0.0764
7,0.1016,0.0689


In [56]:
#для анализа на корреляцию создаем DataFrame, в котором город производства мидий обозначен цифрами: 1 - peterburg, 0 - magadan
city_ls = [] #создаем список маркеров городов
for i in range(len(spb_ls)):
    city_ls.append(1)
for k in range(len(magadan_ls)):
    city_ls.append(0) 
size_ls = spb_ls #создаем список значений размеров мидий
for el in magadan_ls:
    size_ls.append(el)

#создаем DataFrame 
midia_cor_df = pd.DataFrame({'size': size_ls, 'city': city_ls})
midia_cor_df 

,size,city
0,0.0974,1
1,0.1352,1
2,0.0817,1
3,0.1016,1
4,0.0968,1
5,0.1064,1
6,0.1050,1
7,0.1016,1
8,0.1033,0
9,0.0915,0


# ПРОВЕРКА ДАННЫХ НА НОРМАЛЬНОСТЬ

In [57]:
H0 = 'Данные распределены нормально'
Ha = 'Данные не распределены нормально (мы отвергаем H0)'

Установим уровень значимости

In [58]:
alpha = 0.05

# Тест Шапиро-Уилка

In [59]:
from scipy.stats import shapiro

In [60]:
_, p = shapiro(midia_df)
print('p=%.3f' % p)

# Интерпретация 

if p > alpha:
	print(H0)
else:
	print(Ha)

p=0.101
Данные распределены нормально


# Тест Д'Агостино

In [61]:
from scipy.stats import normaltest

In [62]:
_, p = normaltest(midia_df)
#normaltest возвращает двустороннюю вероятность для проверки гипотезы

print('p=%.3f' % p[0])

# Интерпретация 

if p[0] > alpha/2:
	print(H0)
else:
	print(Ha)

p=0.035
Данные распределены нормально


/home/tatiana/anaconda3/lib/python3.8/site-packages/scipy/stats/stats.py:1541: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=8
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "


# ВЫВОДЫ

1. Мы провели тест на нормальность и поняли, что ДАННЫЕ БЛИЗКИ К ГАУСОВСКОМУ РАСПРЕДЕЛЕНИЮ (в данном случае, мы можем проигнорировать результат теста Д'Агостино, поскольку выборка данных слишком мала, чтобы получить достоверный результат).    

2. Мы можем протестировать данные на корреляцию, используя ТЕСТ ПИРСОНА.

3. Применим ПАРАМЕТРИЧЕСКИЙ СРАВНИТЕЛЬНЫЙ ТЕСТ. Зависимая переменная - количественная (средний размер раковины мидий). Группы происходят из разных совокупностей (размер раковин, найденых у городов Санкт-Петербург и Магадан). Значит можно применить зависимый T-TEST)

# КОРРЕЛЯЦИЯ ПИРСОНА

In [63]:
round(midia_cor_df.corr(),2)

,size,city
size,1.00,0.69
city,0.69,1.00


Вывод: Значение коэффициента корреляции 0.69. Между средним размером раковины мидии и городом, в котором мидии выращивают есть сильная положительная связь.

# НЕЗАВИСИМЫЙ t-ТЕСТ

In [64]:
from scipy.stats import ttest_ind

In [65]:
H0 = 'Нет значимой разницы между средним размером раковины мидий выращеных в Санкт-Петербурге и Магадане.'
Ha = 'Есть значимая разница между средним размером раковины мидий выращеных в Санкт-Петербурге и Магадане.'

In [66]:
alpha = 0.05

In [67]:
def t_test(df):
    print('\n' + "*** Результаты независимого T-теста ***")
    test_results = ttest_ind(df['petersburg'], df['magadan'], equal_var=True)

    p = round(test_results[1],2)

    if p>alpha:
        print(f"{p} > {alpha}. Мы не можем отвергнуть нулевую гипотезу. {H0}")
    else:
        print(f"{p} <= {alpha}. Мы отвергаем нулевую гипотезу. {Ha}")

t_test(midia_df)


*** Результаты независимого T-теста ***
0.0 <= 0.05. Мы отвергаем нулевую гипотезу. Есть значимая разница между средним размером раковины мидий выращеных в Санкт-Петербурге и Магадане.


In [68]:
midia_df.mean()

petersburg    0.103212
magadan       0.078012
dtype: float64

# ВЫВОД

Есть ЗНАЧИТЕЛЬНАЯ РАЗНИЦА между средним размером раковины мидий выращеных в Санкт-Петербурге и Магадане. МИДИИ, ВЫРАЩЕННЫЕ В САНКТ-ПЕТЕРБУРГЕ ИМЕЮТ ЗНАЧИТЕЛЬНО БОЛЬШИЙ РАЗМЕР, ЧЕМ МИДИИ, ВЫРАЩЕННЫЕ В МАГАДАНЕ.